<a href="https://colab.research.google.com/github/0xMalhotra/UAS-project/blob/main/UASfinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

FINAL CODE

In [2]:
import cv2
import numpy as np
import math
import glob
import os
import matplotlib.pyplot as plt

# --- 1. CONFIGURATION AND MAPPINGS ---

# Define priority scores for casualties and emergencies
CASUALTY_SCORES = {"Child (Star)": 3, "Triangle": 2, "Square": 1}
EMERGENCY_SCORES = {"red": 3, "yellow": 2, "green": 1}

# Define pad capacities and colors
PAD_CONFIG = {
    "pink": {"capacity": 3, "center": None, "assigned": [], "score_sum": 0, "color_bgr": (255, 0, 255)},
    "blue": {"capacity": 4, "center": None, "assigned": [], "score_sum": 0, "color_bgr": (255, 0, 0)},
    "grey": {"capacity": 2, "center": None, "assigned": [], "score_sum": 0, "color_bgr": (128, 128, 128)}
}

# Define HSV color ranges for detection [Lower_Bound, Upper_Bound]
COLOR_RANGES = {
    # Pad Colors
    "blue": [np.array([90, 80, 50]), np.array([130, 255, 255])],
    "pink": [np.array([140, 80, 100]), np.array([170, 255, 255])],
    "grey": [np.array([0, 0, 50]), np.array([180, 30, 220])],
    # Casualty Colors - Further adjusted ranges and added a broader red range
    "red": [np.array([0, 100, 100]), np.array([10, 255, 255])],
    "red_alt": [np.array([170, 100, 100]), np.array([180, 255, 255])], # Wrap around for red
    "yellow": [np.array([20, 100, 100]), np.array([40, 255, 255])],
    "green": [np.array([35, 80, 50]), np.array([85, 255, 255])]
}

# --- 2. HELPER FUNCTIONS ---

def get_shape_templates():
    templates = {}
    # Star
    star_img = np.zeros((100, 100), dtype=np.uint8)
    pts = np.array([[50, 5], [61, 35], [95, 35], [68, 57], [79, 91], [50, 70], [21, 91], [32, 57], [5, 35], [39, 35]], np.int32)
    cv2.fillPoly(star_img, [pts.reshape((-1, 1, 2))], 255)
    contours, _ = cv2.findContours(star_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    templates["Child (Star)"] = contours[0] if contours else None
    # Circle
    circle_img = np.zeros((100, 100), dtype=np.uint8)
    cv2.circle(circle_img, (50, 50), 45, 255, -1)
    contours, _ = cv2.findContours(circle_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    templates["Circle"] = contours[0] if contours else None
    # Square
    square_img = np.zeros((100, 100), dtype=np.uint8)
    cv2.rectangle(square_img, (10, 10), (90, 90), 255, -1)
    contours, _ = cv2.findContours(square_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    templates["Square"] = contours[0] if contours else None
    # Triangle
    tri_img = np.zeros((100, 100), dtype=np.uint8)
    pts = np.array([[50, 10], [10, 90], [90, 90]], np.int32)
    cv2.fillPoly(tri_img, [pts], 255)
    contours, _ = cv2.findContours(tri_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    templates["Triangle"] = contours[0] if contours else None
    return templates


def get_centroid(contour):
    M = cv2.moments(contour)
    if M["m00"] == 0: return None
    return (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))

def classify_shape(contour, templates):
    min_score, best_shape = float('inf'), "Unknown"
    for shape_name, template_contour in templates.items():
        if template_contour is not None:
            # Add a check for the area ratio to filter out incorrect circle detections
            area = cv2.contourArea(contour)
            template_area = cv2.contourArea(template_contour)
            # Relaxed area ratio threshold slightly
            if template_area > 0 and abs(area / template_area - 1) > 1.0: # Adjusted threshold
                 continue

            score = cv2.matchShapes(contour, template_contour, cv2.CONTOURS_MATCH_I1, 0.0)
            if score < min_score:
                min_score, best_shape = score, shape_name

    # Add an additional check for circularity for better circle detection
    if best_shape == "Circle":
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)
        if perimeter > 0:
            circularity = 4 * np.pi * area / (perimeter ** 2)
            if circularity < 0.65: # Adjusted circularity threshold
                best_shape = "Unknown"

    return best_shape


def euclidean_distance(p1, p2):
    return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

def segment_land_ocean(image):
    # Convert to HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define range for water color (adjust as needed)
    # This is a basic example, more robust methods might be needed for varied images
    lower_blue = np.array([90, 50, 50])
    upper_blue = np.array([130, 255, 255])
    mask_water = cv2.inRange(hsv, lower_blue, upper_blue)

    # Invert mask to get land
    mask_land = cv2.bitwise_not(mask_water)

    # Create colored overlays
    water_overlay = np.zeros_like(image, dtype=np.uint8)
    land_overlay = np.zeros_like(image, dtype=np.uint8)

    # Define colors for land and ocean (e.g., green for land, blue for ocean)
    water_color = (255, 0, 0)  # Blue in BGR
    land_color = (0, 255, 0)   # Green in BGR

    water_overlay[mask_water > 0] = water_color
    land_overlay[mask_land > 0] = land_color

    return land_overlay, water_overlay, mask_water


# --- 3. MAIN PROCESSING FUNCTION ---

def process_image(image_path, templates):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Warning: Could not read image at {image_path}. Skipping.")
        return None
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    casualties = []
    # Create a deep copy to avoid modifying the global config
    pads = {k: v.copy() for k, v in PAD_CONFIG.items()}

    # A. Detect all objects
    casualty_shape_colors = {} # To count unique shape-color combinations

    # Get the water mask
    _, _, mask_water = segment_land_ocean(image)

    for color_name, (lower, upper) in COLOR_RANGES.items():
        # Handle the split red color range
        if color_name == "red_alt":
             mask1 = cv2.inRange(hsv_image, COLOR_RANGES["red"][0], COLOR_RANGES["red"][1])
             mask2 = cv2.inRange(hsv_image, lower, upper)
             mask = cv2.bitwise_or(mask1, mask2)
             current_color_name = "red" # Treat red_alt as red for casualty assignment
        elif color_name != "red": # Process other colors normally, and avoid processing "red" twice
             mask = cv2.inRange(hsv_image, lower, upper)
             current_color_name = color_name
        else:
             continue # Skip the initial "red" entry as it's handled by "red_alt"

        # Apply morphological operations to improve mask - Increased iterations
        kernel = np.ones((3,3),np.uint8)
        mask = cv2.erode(mask,kernel,iterations = 2) # Increased iterations
        mask = cv2.dilate(mask,kernel,iterations = 2) # Increased iterations

        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area < 80: continue # Adjusted minimum area again
            centroid = get_centroid(cnt)
            if not centroid: continue

            shape = classify_shape(cnt, templates)

            if shape == "Circle" and current_color_name in pads:
                pads[current_color_name]["center"] = centroid
            elif shape in CASUALTY_SCORES and current_color_name in EMERGENCY_SCORES: # Check if it's a recognized casualty shape
                # Check if the casualty is in the water based on the mask
                is_in_water = mask_water[centroid[1], centroid[0]] > 0

                cs = CASUALTY_SCORES[shape]
                es = EMERGENCY_SCORES[current_color_name]
                casualties.append({
                    "shape": shape, "color": current_color_name, "center": centroid,
                    "casualty_score": cs, "emergency_score": es, "priority_score": cs * es,
                    "in_water": is_in_water # Add flag for being in water
                })
                # Count unique shape-color combinations
                shape_color_key = f"{shape}-{current_color_name}"
                casualty_shape_colors[shape_color_key] = casualty_shape_colors.get(shape_color_key, 0) + 1


    # B. Assignment Algorithm & Drawing Logic
    # Sort casualties by whether they are in water (water first), then by priority score (descending)
    casualties.sort(key=lambda c: (c["in_water"], c["priority_score"]), reverse=True)

    output_image = image.copy() # Use the original image as the base for drawing

    # Add land/ocean segmentation overlay
    land_overlay, water_overlay, _ = segment_land_ocean(image)
    output_image = cv2.addWeighted(output_image, 1.0, land_overlay, 0.5, 0)
    output_image = cv2.addWeighted(output_image, 1.0, water_overlay, 0.5, 0)


    # Calculate distance matrix - Only for detected casualties
    distance_matrix = {}
    for i, casualty in enumerate(casualties):
        distance_matrix[f"casualty_{i}"] = {}
        for pad_color, pad_info in pads.items():
            if pad_info.get("center"):
                dist = euclidean_distance(casualty["center"], pad_info["center"])
                distance_matrix[f"casualty_{i}"][pad_color] = dist
            else:
                distance_matrix[f"casualty_{i}"][pad_color] = float('inf') # Set distance to infinity if pad not found


    # Assignment based on priority (water first) and then distance
    assigned_casualties_details = {"blue": [], "pink": [], "grey": []}
    unassigned_casualties_count = len(casualties) # Initialize unassigned count

    # Iterate through casualties and assign them to the best available pad
    for casualty in casualties:
        best_pad = None
        min_distance = float('inf')

        # Find the closest pad with available capacity
        for pad_color, pad_info in pads.items():
             if pad_info.get("center") and len(pad_info.get("assigned", [])) < pad_info["capacity"]:
                dist = euclidean_distance(casualty["center"], pad_info["center"])
                if dist < min_distance:
                    min_distance = dist
                    best_pad = pad_color

        if best_pad:
            pads[best_pad]["assigned"].append(casualty)
            pads[best_pad]["score_sum"] += casualty["priority_score"]
            assigned_casualties_details[best_pad].append([casualty["shape"], casualty["color"]])
            unassigned_casualties_count -= 1 # Decrement unassigned count when assigned

            # --- VISUALIZATION DRAWING ---
            pad_center = pads[best_pad]["center"]
            pad_draw_color = pads[best_pad]["color_bgr"]
            # Draw line from casualty to pad
            cv2.line(output_image, casualty["center"], pad_center, pad_draw_color, 2)
            # Draw a circle on the casualty for emphasis
            cv2.circle(output_image, casualty["center"], 15, pad_draw_color, 2)
            # Add text for casualty priority
            cv2.putText(output_image, str(casualty["priority_score"]), (casualty["center"][0] + 20, casualty["center"][1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, pad_draw_color, 2)
        else:
             # Draw a red circle around unassigned casualties
            cv2.circle(output_image, casualty["center"], 20, (0, 0, 255), 2) # Red color for unassigned


    # C. Generate Outputs
    pad_priorities = [pads["blue"]["score_sum"], pads["pink"]["score_sum"], pads["grey"]["score_sum"]]
    total_priority = sum(pad_priorities)
    rescue_ratio = total_priority / (total_priority + (unassigned_casualties_count * 5)) if (total_priority + (unassigned_casualties_count * 5)) > 0 else 0 # Adjusted rescue ratio calculation


    return {
        "output_image": output_image,
        "total_casualties": len(casualties),
        "unique_shape_colors": casualty_shape_colors,
        "distance_matrix": distance_matrix,
        "assigned_details": assigned_casualties_details,
        "pad_priorities": pad_priorities,
        "rescue_ratio": rescue_ratio,
        "unassigned_casualties_count": unassigned_casualties_count # Add unassigned count to output
    }

# --- 4. MAIN EXECUTION ---

if __name__ == "__main__":
    # Create directories if they don't exist
    os.makedirs("input_images", exist_ok=True)
    os.makedirs("output_images", exist_ok=True)

    templates = get_shape_templates()
    # IMPORTANT: Place your 10 input images in a folder named 'input_images'
    # Reverted to process all images in input_images folder
    image_files = glob.glob("input_images/*.png") + glob.glob("input_images/*.jpg")


    if not image_files:
        print("ERROR: No images found in the 'input_images' folder. Please add your images and try again.")

    all_results = []
    for image_file in image_files:
        result = process_image(image_file, templates)
        if result:
            all_results.append({"image_name": os.path.basename(image_file), "rescue_ratio": result["rescue_ratio"]})

            # Save the visual output image
            output_filename = os.path.join("output_images", f"output_{os.path.basename(image_file)}")
            cv2.imwrite(output_filename, result["output_image"])

            print(f"--- Results for {os.path.basename(image_file)} ---")
            print(f"Total Casualties Detected: {result['total_casualties']}")
            print(f"Unassigned Casualties: {result['unassigned_casualties_count']}") # Print unassigned count
            print(f"Unique Shape-Color Combinations Detected: {result['unique_shape_colors']}")
            # print(f"Distance Matrix (Casualties to Pads): {result['distance_matrix']}") # Keep this commented out for cleaner output
            print(f"Assigned Casualties (by pad - [Shape, Color]): {result['assigned_details']}")
            print(f"Pad Priority Sums [Blue, Pink, Grey]: {result['pad_priorities']}")
            print(f"Rescue Ratio (Pr): {result['rescue_ratio']:.2f}\n")


    # Final ranked list of images
    all_results.sort(key=lambda r: r["rescue_ratio"], reverse=True)
    ranked_images = [r["image_name"] for r in all_results]
    print("--- FINAL RANKING ---")
    print(f"Images ranked by Rescue Ratio (descending): {ranked_images}")

--- Results for 10.png ---
Total Casualties Detected: 6
Unassigned Casualties: 2
Unique Shape-Color Combinations Detected: {'Square-yellow': 2, 'Square-green': 4}
Assigned Casualties (by pad - [Shape, Color]): {'blue': [['Square', 'yellow'], ['Square', 'yellow'], ['Square', 'green'], ['Square', 'green']], 'pink': [], 'grey': []}
Pad Priority Sums [Blue, Pink, Grey]: [6, 0, 0]
Rescue Ratio (Pr): 0.38

--- Results for 1.png ---
Total Casualties Detected: 4
Unassigned Casualties: 1
Unique Shape-Color Combinations Detected: {'Square-yellow': 3, 'Square-green': 1}
Assigned Casualties (by pad - [Shape, Color]): {'blue': [], 'pink': [['Square', 'yellow'], ['Square', 'yellow'], ['Square', 'yellow']], 'grey': []}
Pad Priority Sums [Blue, Pink, Grey]: [0, 6, 0]
Rescue Ratio (Pr): 0.55

--- Results for 5.png ---
Total Casualties Detected: 2
Unassigned Casualties: 2
Unique Shape-Color Combinations Detected: {'Triangle-yellow': 1, 'Square-green': 1}
Assigned Casualties (by pad - [Shape, Color]): {'

# Rescue Pad Assignment and Image Analysis

This script processes images containing colored shapes (casualties) and colored circles (rescue pads) to simulate and visualize the assignment of casualties to the nearest available rescue pad based on defined priorities and pad capacities. It also calculates a "Rescue Ratio" to evaluate the effectiveness of the assignment for each image.

## Functionality

1.  **Object Detection**: Identifies different colored shapes (Triangle, Square, Star) as casualties and colored circles (Blue, Pink, Grey) as rescue pads using HSV color range thresholding and contour analysis.
2.  **Shape Classification**: Classifies detected shapes by matching their contours against predefined templates.
3.  **Casualty Prioritization**: Assigns priority scores to casualties based on their shape and color.
4.  **Rescue Pad Assignment**: Assigns casualties to the closest rescue pad with available capacity, prioritizing casualties in water first, then by their priority score.
5.  **Land/Ocean Segmentation**: Segments the image into land and ocean areas for visualization.
6.  **Visualization**: Draws lines connecting assigned casualties to their rescue pads, highlights unassigned casualties, and overlays land/ocean segmentation on the output image.
7.  **Output Metrics**: Calculates and prints the total number of casualties, unassigned casualties, unique shape-color combinations, assigned casualties per pad, pad priority sums, and a Rescue Ratio for each processed image.
8.  **Ranking**: Ranks the processed images based on their Rescue Ratio.

## How to Use

1.  **Prepare your images**: Place the images you want to process in a folder named `input_images` in the same directory as your script. The script currently supports `.png` and `.jpg` files.
2.  **Run the script**: Execute the code cell.
3.  **View Results**:
    *   Output images with visualizations will be saved to a folder named `output_images`.
    *   Detailed results for each image (total casualties, unassigned, unique combinations, assignments, pad sums, rescue ratio) will be printed to the console.
    *   A final ranking of images based on their Rescue Ratio will be printed to the console.

## Configuration

You can adjust the following parameters within the script:

*   `CASUALTY_SCORES`: Modify the priority scores for different shapes.
*   `EMERGENCY_SCORES`: Modify the emergency scores for different colors.
*   `PAD_CONFIG`: Adjust the capacity and colors of the rescue pads.
*   `COLOR_RANGES`: Fine-tune the HSV color ranges for detecting different objects.
*   Minimum contour area thresholds (`if area < 80: continue`): Adjust to filter out smaller noise or include smaller objects.
*   Morphological operation iterations (`iterations = 2`): Adjust to improve mask quality.
*   Shape classification thresholds (`abs(area / template_area - 1) > 1.0`, `circularity < 0.65`): Adjust for better shape detection accuracy.
*   Rescue Ratio calculation formula: Modify the formula in the `process_image` function if needed.

## Dependencies

This script requires the following libraries:

*   `opencv-python`
*   `numpy`
*   `math`
*   `glob`
*   `os`
*   `matplotlib`

You can install them using pip: